## importing necessary libraries:

In [4]:
import pandas as pd
import requests
import json

import psycopg2
import psycopg2.extras as extras
import pytz

## Data Collection from BALLDONTLIE API : 

### PLAYERS DATA :

In [156]:
#call api for "PLAYERS" data : 1) read "final_counters" file to read the last page form where we will retreive data

players_data = []

players_url = "https://www.balldontlie.io/api/v1/players"
per_page = 100
# read final_counter_file content and asign it to counter variable : 

with open("final_counter.txt",'r') as f : 
    final_counter = (json.load(f)["players_final_counter"])
    counter = final_counter + 1

while type(counter) is int : 
    players_call = requests.get(f"{players_url}?per_page={per_page}&page={counter}")
    players_data.extend(players_call.json()["data"])
    
    if players_call.json()["meta"]["next_page"] is not int and len(players_data)>0:
        final_counter = counter
        
    counter = players_call.json()["meta"]["next_page"]

players_df = pd.json_normalize(players_data)

# update final_counter_file value:
with open("final_counter.txt", "w") as f: 
    f.write(f"{{\"players_final_counter\" : {final_counter}}} ")


In [35]:
players_df = players_df[["id", "first_name" , "last_name", "position", "height_feet", "height_inches", "weight_pounds", "team.id", "team.abbreviation", "team.city", "team.conference", "team.division", "team.name", "team.full_name"]]

## Connecting to Postgres DataBase : 

### Inserting Data Into PLAYERS Table :

In [88]:
connection, cursor = None, None

tuples = [tuple(x) for x in players_df.to_numpy()]

cols = ",".join(players_df.columns.to_list()).replace(".", "_")

table_name = "players"

try : 
    connection = psycopg2.connect(user = "postgres", 
                                  password="postgrepass", 
                                  host="localhost", 
                                  port="5432", 
                                  database="nba")
    cursor = connection.cursor()

    create_table_script = f''' CREATE TABLE IF NOT EXISTS {table_name}(
                            id integer,
                            first_name varchar,
                            last_name varchar,
                            position varchar,
                            height_feet float,
                            height_inches float,
                            weight_pounds float,
                            team_id integer,
                            team_abbreviation varchar,
                            team_city varchar,
                            team_conference varchar,
                            team_division varchar,
                            team_name varchar,
                            team_full_name varchar)'''
    
    cursor.execute(create_table_script)
    
    insert_data_script = f''' INSERT INTO players ({cols}) VALUES %s ON CONFLICT DO NOTHING ''' 
     
    # Execute the query with the data using execute_values() method
    extras.execute_values(cursor, insert_data_script, tuples)
    
    connection.commit()
    
except Exception as error:
    print(error)
finally :
    if cursor is not None: 
        cursor.close()
    if connection is not None: 
        connection.close()

